In [1]:
import pandas as pd
from datetime import datetime as dt
from pyspark.sql import SparkSession
from pyspark.sql.functions import  col,date_format,current_date
import calendar

In [2]:
spark=SparkSession.builder.appName('BRONZE TO SILVER').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/06 15:23:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/06 15:23:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
bronze_path = '../DataLake/Bronze/'
silver_path = '../DataLake/Silver/'

### Load data from landing into Spark dataframe

In [4]:
"""LOADING DELTA FILES FROM LANDING """                       
vehicle_df = (spark.read.format('parquet').load(bronze_path).where(f"isNotNull(POSTING_DATE)"))
vehicle_df.limit(10).toPandas()

23/10/06 15:23:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,ID,URL,REGION,REGION_URL,PRICE,YEAR,MANUFACTURER,MODEL,CONDITION,CYLINDERS,...,IMAGE_URL,DESCRIPTION,COUNTY,STATE,LAT,LONG,REMOVAL_DATE,FILE_NAME,RUN_DATE_ID,POSTING_DATE
0,7316755549,https://portland.craigslist.org/clk/ctd/d/atla...,portland,https://portland.craigslist.org,33990,2018,toyota,sienna se minivan 4d,good,6 cylinders,...,https://images.craigslist.org/00Y0Y_j9U914S0C8...,Carvana is the safer way to buy a car During t...,None,or,33.779214,-84.411811,20210528,file:///Users/gaylord/Documents/Athena/DataLak...,20231006,20210504
1,7316793779,https://richmond.craigslist.org/ctd/d/powhatan...,richmond,https://richmond.craigslist.org,6395,2007,toyota,corolla,None,None,...,https://images.craigslist.org/00h0h_ZGDKbo4Fu9...,2007 Toyota Corolla clean car fax with only 79...,None,va,37.5421,-77.9189,20210513,file:///Users/gaylord/Documents/Athena/DataLak...,20231006,20210504
2,7316749635,https://portland.craigslist.org/clk/ctd/d/atla...,portland,https://portland.craigslist.org,24990,2018,volkswagen,golf alltrack tsi s,good,None,...,https://images.craigslist.org/00G0G_1MUyZgXGLO...,Carvana is the safer way to buy a car During t...,None,or,33.779214,-84.411811,20210512,file:///Users/gaylord/Documents/Athena/DataLak...,20231006,20210504
3,7316787141,https://richmond.craigslist.org/ctd/d/richmond...,richmond,https://richmond.craigslist.org,9995,2005,toyota,avalon limited,like new,6 cylinders,...,https://images.craigslist.org/00b0b_8e4BR93Vmx...,2005 Toyota Avalon Limited Stock Number\tA3392...,None,va,37.471588,-77.526226,20210520,file:///Users/gaylord/Documents/Athena/DataLak...,20231006,20210504
4,7316755431,https://portland.craigslist.org/clk/ctd/d/atla...,portland,https://portland.craigslist.org,15990,2019,hyundai,accent se sedan 4d,good,None,...,https://images.craigslist.org/00z0z_bzdFcf193F...,Carvana is the safer way to buy a car During t...,None,or,33.779214,-84.411811,20210525,file:///Users/gaylord/Documents/Athena/DataLak...,20231006,20210504
5,7316715449,https://richmond.craigslist.org/ctd/d/richmond...,richmond,https://richmond.craigslist.org,2999,2006,honda,civic,None,None,...,https://images.craigslist.org/00c0c_lhcEpFZnVi...,Runs great 200k 5 speed manual sunroof 571208...,None,va,37.4638,-77.398,20210530,file:///Users/gaylord/Documents/Athena/DataLak...,20231006,20210504
6,7316749563,https://portland.craigslist.org/clk/ctd/d/atla...,portland,https://portland.craigslist.org,35990,2020,infiniti,qx50 pure sport utility,good,None,...,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,None,or,33.779214,-84.411811,20210507,file:///Users/gaylord/Documents/Athena/DataLak...,20231006,20210504
7,7316914795,https://richmond.craigslist.org/ctd/d/richmond...,richmond,https://richmond.craigslist.org,6500,2009,bmw,328,None,6 cylinders,...,https://images.craigslist.org/00c0c_fr2Fck9jpm...,2009 BMW 328i Coupe AWD strait 6 Automatic wit...,None,va,37.502481,-77.61446,20210529,file:///Users/gaylord/Documents/Athena/DataLak...,20231006,20210504
8,7316749523,https://portland.craigslist.org/clk/ctd/d/atla...,portland,https://portland.craigslist.org,38990,2020,ram,1500 quad cab lone star,good,6 cylinders,...,https://images.craigslist.org/00K0K_iXnibMEVpj...,Carvana is the safer way to buy a car During t...,None,or,33.779214,-84.411811,20210525,file:///Users/gaylord/Documents/Athena/DataLak...,20231006,20210504
9,7316946241,https://richmond.craigslist.org/ctd/d/richmond...,outer banks,https://outerbanks.craigslist.org,2499,2010,chevrolet,malibu,None,4 cylinders,...,https://images.craigslist.org/00J0J_bisNX3KquA...,Runs great 200k aux everything works current ...,None,nc,37.4638,-77.398,20210517,file:///Users/gaylord/Documents/Athena/DataLak...,20231006,20210504


### Select only required columns

In [5]:
vehicle_df = vehicle_df.select('ID','REGION','PRICE','YEAR','MANUFACTURER','MODEL','CONDITION','CYLINDERS','FUEL',
                              'ODOMETER','TITLE_STATUS','TRANSMISSION','VIN','DRIVE','SIZE','TYPE','PAINT_COLOR','COUNTY','STATE',
                              'POSTING_DATE','REMOVAL_DATE','FILE_NAME')

### Add Watermark Column

In [6]:
vehicle_df = (vehicle_df.withColumn('RUN_DATE_ID', date_format(current_date(),"yyyyMMdd").cast('int')))

### Show 5 records as Pandas DataFrame for better visibility

In [7]:
vehicle_df.limit(5).toPandas().head()

,ID,REGION,PRICE,YEAR,MANUFACTURER,MODEL,CONDITION,CYLINDERS,FUEL,ODOMETER,...,DRIVE,SIZE,TYPE,PAINT_COLOR,COUNTY,STATE,POSTING_DATE,REMOVAL_DATE,FILE_NAME,RUN_DATE_ID
0,7316755549,portland,33990,2018,toyota,sienna se minivan 4d,good,6 cylinders,gas,16607.0,...,fwd,None,van,white,None,or,20210504,20210528,file:///Users/gaylord/Documents/Athena/DataLak...,20231006
1,7316793779,richmond,6395,2007,toyota,corolla,None,None,gas,79512.0,...,None,mid-size,sedan,white,None,va,20210504,20210513,file:///Users/gaylord/Documents/Athena/DataLak...,20231006
2,7316749635,portland,24990,2018,volkswagen,golf alltrack tsi s,good,None,other,19099.0,...,None,None,wagon,red,None,or,20210504,20210512,file:///Users/gaylord/Documents/Athena/DataLak...,20231006
3,7316787141,richmond,9995,2005,toyota,avalon limited,like new,6 cylinders,gas,100304.0,...,fwd,full-size,sedan,None,None,va,20210504,20210520,file:///Users/gaylord/Documents/Athena/DataLak...,20231006
4,7316755431,portland,15990,2019,hyundai,accent se sedan 4d,good,None,gas,31913.0,...,fwd,None,sedan,red,None,or,20210504,20210525,file:///Users/gaylord/Documents/Athena/DataLak...,20231006


### Generate Dates from the dataframe to be used for partitioning when writing into SILVER layer

In [8]:
unique_dates = vehicle_df.select("POSTING_DATE").distinct().rdd.flatMap(lambda x: x).collect()

dates_processed = [] 
for dates_ in unique_dates : 
    dates_processed.append(dates_)

#Create a comma separated list
dates_processed_str = ",".join(str(i) for i in dates_processed)

partition_predicate = f"POSTING_DATE IN ({dates_processed_str}) " 

In [9]:
#this is to force caching process
print(vehicle_df.count())

362915


### Load into Silver

In [10]:
"""WRITTING TO SILVER LAYE"""
(vehicle_df.write.format("parquet")
 .partitionBy("POSTING_DATE")
 .option("replaceWhere",  f"{partition_predicate}")
 .mode("overwrite").save(silver_path))

vehicle_df.unpersist(blocking=True)

23/10/06 15:23:48 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/10/06 15:23:48 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/10/06 15:23:48 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/10/06 15:23:48 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/10/06 15:23:48 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
23/10/06 15:23:48 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 56.31% for 12 writers
23/10/06 15:23:49 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,

DataFrame[ID: string, REGION: string, PRICE: string, YEAR: int, MANUFACTURER: string, MODEL: string, CONDITION: string, CYLINDERS: string, FUEL: string, ODOMETER: string, TITLE_STATUS: string, TRANSMISSION: string, VIN: string, DRIVE: string, SIZE: string, TYPE: string, PAINT_COLOR: string, COUNTY: string, STATE: string, POSTING_DATE: int, REMOVAL_DATE: string, FILE_NAME: string, RUN_DATE_ID: int]